In [ ]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path

import adcraft.baselines.interpolated_expectations as ie
import adcraft.gymnasium_kw_env as kw_sim
from adcraft.experiment_utils.experiment_quantiles import (
    make_experiment_quantiles, load_experiment_quantiles, make_bctr_experiment_quantiles, load_bctr_experiment_quantiles, make_vol_bctr_experiment_quantiles, load_vol_bctr_experiment_quantiles
)
from adcraft.experiment_utils.experiment_metrics import (
    get_implicit_kw_bid_cpc_impressions, get_max_expected_bid_profits, compute_AKNCP, compute_NCP
)
import adcraft.visualization.jupyter_functions as bvis


In [ ]:
def cache_tensors_to_floats(cache):
    cache["ave_rpc"] = float(cache["ave_rpc"])
    cache["ave_sctr"] = float(cache["ave_sctr"])
    cache["ave_clicks"] = {k: [float(v[0]), v[1]] for k, v in cache["ave_clicks"].items()}

def run_zero_margin_agent(agent, env, irs, cpcs, budget = 100000, vis: bool = False):
    previous_observation, info = env.reset()
    previous_action = {"budget": 0.0, "keyword_bids": 0.01 + np.zeros((env.num_keywords,))}
    truncated, terminated = False, False

    kw_profits = []
    ideal_profits = []
    while not (terminated or truncated):
        # agent update
        agent.update_all_caches(previous_action, previous_observation)
        for i in range(env.num_keywords):
            cache_tensors_to_floats(agent.caches[i])

        # compute next action
        action = agent.sample_action()

        # compute ideal profits for this timestep
        ideal_profit = []
        for kw_index, kw_params in enumerate(env.keyword_params):
            max_exp_profit, positive_proportion, _ = get_max_expected_bid_profits(
                kw_params, cpcs[kw_index], irs[kw_index]
            )
            ideal_profit.append(max_exp_profit)
        ideal_profits.append(ideal_profit)

        # take step
        previous_observation, reward, terminated, truncated, info = env.step(
            action = {"budget": budget, "keyword_bids": action["keyword_bids"]}
        )  
        previous_action = action

        # tabulate outcomes
        kw_profits.append(previous_observation['revenue'] - previous_observation['cost'])
        
        # show running results if needed
        if vis:
            bvis.show_keyword_profits(kw_profits, bids)
            bvis.show_cumulative_rewards(rewards)
    return np.array(kw_profits), np.array(ideal_profits)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def run_sparsity_experiments(mean_volumes, conversion_rates, num_keywords=100, time_steps=60):
    """Runs the volume-conversion rate heatmap experiments for set of mean_volumes and conversion_rates.
    
    Requires that experiment_csvs and experiment_results directories exist on current path.
    """
    for volume in mean_volumes:
        for cvr in conversion_rates:
            env = kw_sim.bidding_sim_creator(
                env_config=dict(
                    keyword_config={
                        "outer_directory": Path.cwd().as_posix()+"/experiment_csvs/",
                        "mean_volume": volume,
                        "conversion_rate": cvr,
                        "make_quant_func": make_experiment_quantiles,
                        "load_quant_func": load_experiment_quantiles
                    },
                    num_keywords = num_keywords,
                    max_days = time_steps,
                    updater_params = [["vol", 0.03], ["ctr", 0.03], ["cvr", 0.03]],
                    updater_mask = None
                ))
            # RUN EXPERIMENT ON ENV
            results_dir = str(Path.cwd().as_posix())+f"/experiment_results/{volume}_{cvr}/"
            if not os.path.isdir(results_dir):
                os.mkdir(results_dir)
                e_s = 5
                a_s = 0
            else:
                files = os.listdir(results_dir)
                if files:
                    e_s = max([int(fname[0]) for fname in files])
                    a_s = max([int(fname[2]) for fname in files])+1
                else:
                    e_s = 5
                    a_s = 0
                    
            for env_seed in range(e_s, 9):
                print(' ' + str(env_seed), end = ' ')
                for agent_seed in range(a_s, 4):
                    env.reset(seed=env_seed)
                    allowed_bids = np.arange(0.01, 3.00, 0.01)
                    irs, cpcs = [],[]
                    for kw in env.keywords:
                        ir, cpc = get_implicit_kw_bid_cpc_impressions(kw, allowed_bids)
                        irs.append(ir)
                        cpcs.append(cpc)
                    agent = ie.NaiveZeroMarginStrategy(
                        env.num_keywords, 
                        default_expected_revenue_per_conversion=1.0, 
                        seed=agent_seed
                    )
                    kw_profits, ideal_profits = run_zero_margin_agent(agent, env, irs, cpcs)
                    np.savez(results_dir+f"{env_seed}_{agent_seed}.npz", kw_profits=kw_profits, ideal_profits=ideal_profits)
                    print("." + str(agent_seed), end = '')
                a_s = 0
            print(f"vol, cvr: ({volume}, {cvr}) done!")

In [ ]:
mean_volumes = [2**k for k in range(10+1)]
conversion_rates = np.linspace(0.01, 1.0, 10)

run_sparsity_experiments(mean_volumes, conversion_rates, num_keywords=100, time_steps=60)

In [ ]:
def run_bctr_sparsity_experiments(clickthrough_rates, conversion_rates, num_keywords=100, time_steps=60):
    """Run the clickthrough rate-conversion rate heatmap experiments for set of clickthrough-rates and conversion_rates.
    
    Requires that bctr_sctr_experiment_csvs and bctr_sctr_experiment_results directories exist on current path.
    """
    for ctr in clickthrough_rates:
        for cvr in conversion_rates:
            env = kw_sim.bidding_sim_creator(
                env_config=dict(
                    keyword_config={
                        "outer_directory": Path.cwd().as_posix()+"/bctr_sctr_experiment_csvs/",
                        "clickthrough_rate": ctr,
                        "conversion_rate": cvr,
                        "make_quant_func": make_bctr_experiment_quantiles,
                        "load_quant_func": load_bctr_experiment_quantiles
                    },
                    num_keywords = num_keywords,
                    max_days = time_steps,
                    updater_params = [["vol", 0.03], ["ctr", 0.03], ["cvr", 0.03]],
                    updater_mask = None
                ))
            # RUN EXPERIMENT ON ENV
            results_dir = str(Path.cwd().as_posix())+f"/bctr_sctr_experiment_results/{ctr}_{cvr}/"
            if not os.path.isdir(results_dir):
                os.mkdir(results_dir)
                e_s = 5
                a_s = 0
            else:
                files = os.listdir(results_dir)
                if files:
                    e_s = max([int(fname[0]) for fname in files])
                    # print(e_s)
                    a_s = max([int(fname[2]) for fname in files])+1
                    # print(a_s+1)
                else:
                    e_s = 5
                    a_s = 0
                    
            for env_seed in range(e_s, 9):
                print(' ' + str(env_seed), end = '')
                for agent_seed in range(a_s, 4):
                    env.reset(seed=env_seed)
                    allowed_bids = np.arange(0.01, 3.00, 0.01)
                    irs, cpcs = [],[]
                    for kw in env.keywords:
                        ir, cpc = get_implicit_kw_bid_cpc_impressions(kw, allowed_bids)
                        irs.append(ir)
                        cpcs.append(cpc)
                    agent = ie.NaiveZeroMarginStrategy(
                        env.num_keywords, 
                        default_expected_revenue_per_conversion=1.0, 
                        seed=agent_seed
                    )
                    kw_profits, ideal_profits = run_zero_margin_agent(agent, env, irs, cpcs)
                    np.savez(results_dir+f"{env_seed}_{agent_seed}.npz", kw_profits=kw_profits, ideal_profits=ideal_profits)
                    print("." + str(agent_seed), end = '')
                a_s = 0
            print(f" ctr, cvr: ({ctr}, {cvr}) done!")

In [ ]:
clickthrough_rates = np.linspace(0.01, 0.8, 10)
conversion_rates = np.linspace(0.01, 0.8, 10)

run_bctr_sparsity_experiments(clickthrough_rates, conversion_rates, num_keywords=100, time_steps=60)

In [ ]:
def run_vol_bctr_sparsity_experiments(mean_volumes, clickthrough_rates, num_keywords=100, time_steps=60):
     """Run the volume-clickthrough rate heatmap experiments for set of mean_volumes and clickthrough-rates.
    
    Requires that vol_bctr_experiment_csvss and vol_bctr_experiment_resultss directories exist on current path.
    """
    for volume in mean_volumes:
        for ctr in clickthrough_rates:
            env = kw_sim.bidding_sim_creator(
                env_config=dict(
                    keyword_config={
                        "outer_directory": Path.cwd().as_posix()+"/vol_bctr_experiment_csvs/",
                        "mean_volume": volume,
                        "clickthrough_rate": ctr,
                        "make_quant_func": make_vol_bctr_experiment_quantiles,
                        "load_quant_func": load_vol_bctr_experiment_quantiles
                    },
                    num_keywords = num_keywords,
                    max_days = time_steps,
                    updater_params = [["vol", 0.03], ["ctr", 0.03], ["cvr", 0.03]],
                    updater_mask = None
                ))
            # RUN EXPERIMENT ON ENV
            results_dir = str(Path.cwd().as_posix())+f"/vol_bctr_experiment_results/{volume}_{ctr}/"
            if not os.path.isdir(results_dir):
                os.mkdir(results_dir)
                e_s = 5
                a_s = 0
            else:
                files = os.listdir(results_dir)
                if files:
                    e_s = max([int(fname[0]) for fname in files])
                    # print(e_s)
                    a_s = max([int(fname[2]) for fname in files])+1
                    # print(a_s+1)
                else:
                    e_s = 5
                    a_s = 0
                    
            for env_seed in range(e_s, 9):
                print(' ' + str(env_seed), end = '')
                for agent_seed in range(a_s, 4):
                    env.reset(seed=env_seed)
                    allowed_bids = np.arange(0.01, 3.00, 0.01)
                    irs, cpcs = [],[]
                    for kw in env.keywords:
                        ir, cpc = get_implicit_kw_bid_cpc_impressions(kw, allowed_bids)
                        irs.append(ir)
                        cpcs.append(cpc)
                    agent = ie.NaiveZeroMarginStrategy(
                        env.num_keywords, 
                        default_expected_revenue_per_conversion=1.0, 
                        seed=agent_seed
                    )
                    kw_profits, ideal_profits = run_zero_margin_agent(agent, env, irs, cpcs)
                    np.savez(results_dir+f"{env_seed}_{agent_seed}.npz", kw_profits=kw_profits, ideal_profits=ideal_profits)
                    print("." + str(agent_seed), end = '')
                a_s = 0
            print(f" vol, ctr: ({volume}, {ctr}) done!")

In [ ]:
mean_volumes = [2**k for k in range(10+1)]
clickthrough_rates = np.linspace(0.01, 1.0, 10)

run_vol_bctr_sparsity_experiments(mean_volumes, clickthrough_rates, num_keywords=100, time_steps=60)